In [155]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [156]:
def populate_data(csv_name, test_percent=0.3):
    df = pd.read_csv(csv_name)

    train, test = train_test_split(df, test_size=test_percent)

    return train, test


In [157]:
def system_entropy(df, target_column):
    count_dict = dict(df[target_column].value_counts())
    unique_vals = (list(set(count_dict.keys())))
    total_len = len(df[target_column])

    sys_entropy = 0
    for value in unique_vals:
        sys_entropy = sys_entropy + (-(count_dict[value] / total_len) * np.log2((count_dict[value] / total_len)))
    return sys_entropy

In [158]:
def entropy(df, target_column, query_column):
    tiny = 0.0000000000001

    list1 = df.groupby([query_column, target_column]).size().reset_index(name="combo_count")
    list2 = df.groupby([query_column]).size().reset_index(name="var_count")

    list1['var_count'] = list1[query_column].map(dict(list2[[query_column, 'var_count']].values))

    list1['entropy'] = -(list1['combo_count'] / (list1['var_count'] + tiny)) * \
                   np.log2((list1['combo_count'] / (list1['var_count'] + tiny)))

    return np.abs(np.sum(list1['entropy']))

In [159]:
def max_information_gain(df, target_column):
    col_list = list(df.columns)
    col_list.remove(target_column)

    column_entropy = {}
    for col in col_list:
        column_entropy[col] = entropy(df, target_column, col)
    max_entropy_key = max(column_entropy, key=column_entropy.get)
    return max_entropy_key, column_entropy[max_entropy_key]

In [160]:
def build_tree(df, target_column, tree_depth=0, max_depth=6):
    tree_depth = tree_depth + 1
    max_ent_key, max_ent_val = max_information_gain(df, target_column)
    max_ent_vals = np.unique(df[max_ent_key])
    tree_struct = {}
    for value in max_ent_vals:
        choice_name = max_ent_key + "_" + str(value)
        #print("choicename", choice_name)
        # return
        only_val = df[df[max_ent_key] == value].reset_index(drop=True)
        #only_val = only_val.drop(max_ent_key, axis=1)
        #print("onlyvals", only_val)
        # print("dropping", max_ent_key)
        # print(only_val)
        # break
        target_vals, target_counts = np.unique(only_val[target_column],return_counts=True)
        if tree_depth >= max_depth:
            index_max = max(range(len(target_counts)), key=target_counts.__getitem__)
            choice = target_vals[index_max]
            #print("depth vote", choice)
            # print("target_vals", target_vals)
            # print("target_counts", target_counts)
            # #print('dataframe', df)
            # print("target_column", target_column)
            tree_struct[choice_name] = choice
        elif len(target_counts) == 1:
            #print("non depth vote", target_vals[0])
            tree_struct[choice_name] = target_vals[0]
        else:
            tree_struct[choice_name] = build_tree(only_val, target_column, tree_depth, max_depth)

    return tree_struct

In [161]:
import collections
def get_tree_vote_list(tree):
    vote_list = []
    if isinstance(tree, dict):
        for key in tree.keys():
            sub_votes = get_tree_vote_list(tree[key])
            if isinstance(sub_votes, list):
                for vote in sub_votes:
                    vote_list.append(vote)
            else:
                vote_list.append(sub_votes)
    elif isinstance(tree, str):
        vote_list.append(tree)
    if len(vote_list) < 1:
        print("votelist tree", tree)
    return vote_list

def get_tree_vote(tree):
    vote_list = get_tree_vote_list(tree)
    occurrences = collections.Counter(vote_list)
    max_key = max(occurrences, key=occurrences.get)
    return max_key


In [162]:
def search_tree(row_dict, tree_to_search, search_depth=0):
    row_keys = []
    for entry in row_dict.keys():
        new_key = entry + "_" +  str(row_dict[entry])
        row_keys.append(new_key)

    if not isinstance(tree_to_search, dict):
        # print("Found Key")
        return tree_to_search

    key_list = list(tree_to_search.keys())
    found_key = False
    # print("Key list:", key_list)
    # print("Row Keys:", row_keys)
    for key in row_keys:
        #print("key:", key)
        if key in key_list:
            return search_tree(row_dict, tree_to_search[key],
                               search_depth=(search_depth+1))
    if not found_key:
        #print("Didn't find key", tree_to_search)
        return get_tree_vote(tree_to_search)


In [163]:
def max_depth(query_tree):
    max_num = 0
    if isinstance(query_tree, dict):
        for key in query_tree.keys():
            key_depth = max_depth(query_tree[key])
            if max_num < key_depth:
                max_num = key_depth
        max_num = max_num + 1
        return max_num
    else:
        return 0

In [164]:
def get_test_results(test_data, test_tree):
    t = 'acc'
    f = 'unacc'
    tn = 0
    tp = 0
    fn = 0
    fp = 0
    broken = 0

    for index, row in test_data.iterrows():
        answer = search_tree(dict(row), test_tree)
        if answer ==  row['class'] and answer == t:
            tp = tp + 1
        elif answer ==  row['class'] and answer == f:
            tn = tn + 1
        elif answer !=  row['class'] and answer == t:
            fp = fp + 1
        elif answer !=  row['class'] and answer == f:
            fn = fn + 1
        else:
            broken = broken + 1
    total_right = tp + tn
    total_wrong = fp + fn
    total_percent = round((total_right / (total_wrong + total_right)) * 100, 2)
    #print("\t\tTP:", tp, " | TN:", tn, " | FP:", fp, " | FN:", fn, " | Total:", total_percent)
    return tp, tn, fp, fn, total_percent

In [173]:
depths_to_try = [10]
train_test_to_try = [0.25]
needs_build = True
target_column = 'class'

import time
total_start = time.time()

best_tree_set = {}
from sklearn.model_selection import KFold
kf = KFold(n_splits = 8, shuffle = True, random_state = 2)
train_df, test_df = populate_data('car_evaluation.csv', test_percent=0.25)

In [174]:
for ttp in train_test_to_try:
    ttp_start = time.time()
    train_df, validation_df = populate_data('car_evaluation.csv', test_percent=ttp)
    best_tree_acc = 0
    best_tree = None
    fold_num = 0
    for train_index, test_index in kf.split(train_df):
        print("\t\tFold Number:", fold_num)
        fold_num = fold_num + 1
        train_set = train_df.iloc[train_index]
        test_set = train_df.iloc[test_index]
        for depth_size in depths_to_try:
            depth_start = time.time()
            print("\t\t\tTrying depth:", depth_size)
            if needs_build:
                usable_tree = build_tree(train_set, target_column, max_depth=depth_size)
            tp, tn, fp, fn, total_percent = get_test_results(test_set, usable_tree)
            print("\t\t\tTP:", tp, " | TN:", tn, " | FP:", fp, " | FN:", fn, " | Total:", total_percent)
            if total_percent > best_tree_acc:
                best_tree = usable_tree
            depth_end = time.time()
            print("\t\t\tDepth took", (depth_end-depth_start), "seconds to train")
    tp, tn, fp, fn, total_percent = get_test_results(validation_df, usable_tree)
    ttp_end = time.time()

    print("*****************************")
    print("Train Test Split:", ttp)
    print("\tValidation Scores - TP:", tp, " | TN:", tn, " | FP:", fp, " | FN:", fn, " | Total:", total_percent)
    print("\tTTS took", (ttp_end-ttp_start), "seconds to train")
    print("*****************************")

    best_tree_set[ttp] = best_tree
        #print("\t\tAnswer List:", answer_list)
        # print("Right:", total_right)
        # print("Wrong:", total_wrong)
        #print("\t\t", depth_size, "percentage correct:", )
print("*****************************")
total_end = time.time()
print("Entire run took:", (total_end - total_start), "seconds")

		Fold Number: 0
			Trying depth: 10
			TP: 23  | TN: 76  | FP: 32  | FN: 19  | Total: 66.0
			Depth took 45.61134576797485 seconds to train
		Fold Number: 1
			Trying depth: 10
			TP: 21  | TN: 79  | FP: 29  | FN: 19  | Total: 67.57
			Depth took 49.0059769153595 seconds to train
		Fold Number: 2
			Trying depth: 10
			TP: 19  | TN: 87  | FP: 34  | FN: 18  | Total: 67.09
			Depth took 46.24462556838989 seconds to train
		Fold Number: 3
			Trying depth: 10
			TP: 22  | TN: 88  | FP: 20  | FN: 24  | Total: 71.43
			Depth took 49.184141397476196 seconds to train
		Fold Number: 4
			Trying depth: 10
			TP: 29  | TN: 89  | FP: 16  | FN: 21  | Total: 76.13
			Depth took 47.79323744773865 seconds to train
		Fold Number: 5
			Trying depth: 10
			TP: 21  | TN: 78  | FP: 39  | FN: 11  | Total: 66.44
			Depth took 47.51473951339722 seconds to train
		Fold Number: 6
			Trying depth: 10
			TP: 24  | TN: 92  | FP: 21  | FN: 16  | Total: 75.82
			Depth took 53.86616587638855 seconds to train
		Fold 

In [ ]:
# QUESTION 2
# The best accuracy happened with a tree depth of 10 and a
# train/test split of 75:25 (This is with leaving 10% out for validation)
# The Classification matrix for each of the train/test/validation follows
# Depth: 10
# Train Test Split: 0.25
# Train:
#   Total: 72.6% Correct
# 	TP: 65  | TN: 237  | FP: 71  | FN: 43  |
#
# Test:
#   Total: 72.48% Correct
# 	TP: 22  | TN: 86  | FP: 26  | FN: 15  |
#
# Validation
#   Total: 73.68% Correct
# 	TP: 32  | TN: 115  | FP: 34  | FN: 18  |

In [172]:
print(len(best_tree_set))
for tree in best_tree_set:
    print("Next Tree")
    tp, tn, fp, fn, total_percent = get_test_results(train_df, tree)
    print("\tTrain Set Scores - TP:", tp, " | TN:", tn, " | FP:", fp, " | FN:", fn, " | Total:", total_percent)


0


In [179]:
# QUESTION 3 #
# In looking at the results of the printed out tree, it seems like
# Doors and  maintenance are the two highest information gain items
# They not only come first in the tree structure, but also re-appear
# further down in the tree when column  re-use is allowed
def print_layer(layer_dict, tab_string=""):
    if not isinstance(layer_dict, dict):
        print(layer_dict)
        return
    tab_string = tab_string + "\t"
    for layer in layer_dict.keys():
        if isinstance(layer_dict[layer], dict):
            print(tab_string, layer, ":", list(layer_dict[layer].keys()))
            print_layer(layer_dict[layer], tab_string)
        else:
            print(tab_string, layer, ":", layer_dict[layer])


print_layer(best_tree_set)

	 0.25 : ['doors_2', 'doors_3', 'doors_4', 'doors_5']
		 doors_2 : ['maint_high', 'maint_low', 'maint_med', 'maint_vhigh']
			 maint_high : ['buying_high', 'buying_low', 'buying_med', 'buying_vhigh']
				 buying_high : ['lug_boot_big', 'lug_boot_med', 'lug_boot_small']
					 lug_boot_big : ['persons_2', 'persons_4', 'persons_5']
						 persons_2 : unacc
						 persons_4 : ['buying_high']
							 buying_high : ['buying_high']
								 buying_high : ['buying_high']
									 buying_high : ['buying_high']
										 buying_high : ['buying_high']
											 buying_high : acc
						 persons_5 : ['buying_high']
							 buying_high : ['buying_high']
								 buying_high : ['buying_high']
									 buying_high : ['buying_high']
										 buying_high : ['buying_high']
											 buying_high : acc
					 lug_boot_med : ['persons_2', 'persons_4', 'persons_5']
						 persons_2 : unacc
						 persons_4 : ['buying_high']
							 buying_high : ['buying_high']
								 buying_high : ['buying_high']
